In [ ]:
from astropy.io import fits

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt

# Going through the tutorial here: https://prospect.readthedocs.io/en/latest/quickstart.html

In [ ]:
import fsps
import dynesty
import sedpy
import h5py, astropy
import numpy as np

In [ ]:
from astroquery.sdss import SDSS
from astropy.coordinates import SkyCoord
bands = "ugriz"
mcol = [f"cModelMag_{b}" for b in bands]
ecol = [f"cModelMagErr_{b}" for b in bands]
cat = SDSS.query_crossid(SkyCoord(ra=204.46376, dec=35.79883, unit="deg"), 
                         data_release=16, photoobj_fields=mcol + ecol + ["specObjID"])
shdus = SDSS.get_spectra(plate=2101, mjd=53858, fiberID=220)[0]
assert int(shdus[2].data["SpecObjID"][0]) == cat[0]["specObjID"]

In [ ]:
from sedpy.observate import load_filters
from prospect.utils.obsutils import fix_obs

filters = load_filters([f"sdss_{b}0" for b in bands])
maggies = np.array([10**(-0.4 * cat[0][f"cModelMag_{b}"]) for b in bands])
magerr = np.array([cat[0][f"cModelMagErr_{b}"] for b in bands])
magerr = np.clip(magerr, 0.05, np.inf)

obs = dict(wavelength=None, spectrum=None, unc=None, redshift=shdus[2].data[0]["z"],
           maggies=maggies, maggies_unc=magerr * maggies / 1.086, filters=filters)
obs = fix_obs(obs)

In [ ]:
from prospect.models.templates import TemplateLibrary
from prospect.models import SpecModel
model_params = TemplateLibrary["parametric_sfh"]
model_params.update(TemplateLibrary["nebular"])
model_params["zred"]["init"] = obs["redshift"]

model = SpecModel(model_params)
assert len(model.free_params) == 5
print(model)

In [ ]:
noise_model = (None, None)

In [ ]:
from prospect.sources import CSPSpecBasis
sps = CSPSpecBasis(zcontinuous=1)
print(sps.ssp.libraries)

In [ ]:
current_parameters = ",".join([f"{p}={v}" for p, v in zip(model.free_params, model.theta)])
print(current_parameters)
spec, phot, mfrac = model.predict(model.theta, obs=obs, sps=sps)
print(phot / obs["maggies"])

In [ ]:
from prospect.fitting import lnprobfn, fit_model
fitting_kwargs = dict(nlive_init=400, nested_method="rwalk", nested_posterior_thresh=0.05)
output = fit_model(obs, model, sps, optimize=False, dynesty=True, lnprobfn=lnprobfn, noise=noise_model, **fitting_kwargs)
result, duration = output["sampling"]

In [ ]:
from prospect.io import write_results as writer
hfile = "quickstart_dynesty_mcmc.h5"
writer.write_hdf5(hfile, {}, model, obs,
                 output["sampling"][0], None,
                 sps=sps,
                 tsample=output["sampling"][1],
                 toptimize=0.0)


# Using Taskfarmer

In [1]:
import pandas as pd
import csv

In [7]:
galaxies = pd.read_csv("../data/first_query.txt", nrows=1000)

In [8]:
shft_cmd = "shifter --image=audreykoz/gradient:latest" \
" --volume='/global/cscratch1/sd/eramey16/data/testrun:/gradient_boosted/exports'" \
" /opt/conda/bin/python /gradient_boosted/classify.py -r {} -d {}"

In [9]:
galaxies['run'] = [shft_cmd.format(a,b) for a,b in zip(galaxies.ra, galaxies.dec)]

In [10]:
galaxies

,ls_id,ra,dec,run
0,8796115184194014,149.285304,1.345116,shifter --image=audreykoz/gradient:latest --vo...
1,8796115184193402,149.273901,1.313647,shifter --image=audreykoz/gradient:latest --vo...
2,8796115184193181,149.291823,1.303267,shifter --image=audreykoz/gradient:latest --vo...
3,8796115278499067,149.127949,1.530269,shifter --image=audreykoz/gradient:latest --vo...
4,8796115278498990,149.121014,1.526591,shifter --image=audreykoz/gradient:latest --vo...
...,...,...,...,...
995,8796115278693161,149.765329,1.415705,shifter --image=audreykoz/gradient:latest --vo...
996,8796115278693010,149.784284,1.406019,shifter --image=audreykoz/gradient:latest --vo...
997,8796115278693406,149.765189,1.427825,shifter --image=audreykoz/gradient:latest --vo...
998,8796115278693488,149.783395,1.435049,shifter --image=audreykoz/gradient:latest --vo...


In [11]:
galaxies['run'].to_csv('../data/tasks.txt', sep='\t', header=False, index=False, quoting=3)

In [38]:
galaxies = pd.read_csv("../data/masterlens.tsv", skiprows=1, sep='\t')

In [41]:
galaxies.columns = galaxies.columns.str.replace('"', '')
galaxies.columns = galaxies.columns.str.strip()
# galaxies.apply(lambda s: s.str.replace('"', ""))
# galaxies.apply(lambda s: s.str.strip())
galaxies.columns

Index(['# system_name', 'discovery_date', 'alternate_name', 'kind_acronym',
       'discovery_acronym', 'reference_identifier', 'ra_hrs', 'ra_mins',
       'ra_secs', 'ra_coord', 'dec_degrees', 'dec_arcmin', 'dec_arcsec',
       'dec_coord', 'lensgrade', 'number_images', 'theta_e', 'theta_e_err',
       'theta_e_quality', 'z_lens', 'z_lens_err', 'z_lens_quality', 'z_source',
       'z_source_err', 'z_source_quality', 'vdisp', 'vdisp_err', 'filter_lens',
       'filter_source', 'fluxes', 'mag_lens', 'mag_source', 'morphology',
       'has_sdss', 'sdss_link', 'has_adsabs', 'adsabs_link', 'apod_link',
       'vett_status', 'released_status', 'hidden_status', 'vetted', 'released',
       'hidden', 'graphic_status', 'equinox', 'sdss_ObjID', 'sdss_specObjID',
       'time_delay0', 'time_delay1', 'lens_name'],
      dtype='object')

In [48]:
radec = galaxies[['ra_coord', 'dec_coord']]
radec.columns = ['ra', 'dec']
radec.to_csv('coords_A.csv', index=False)

In [33]:
galaxies.iloc[0]['discovery_date']

' "2008-08-01"'

In [24]:
galaxies = pd.read_csv('../data/lensed/ML_C/coords_C.csv')
galaxies.columns = [x.lower() for x in galaxies.columns]

In [25]:
radec = galaxies[['ra', 'dec']]

In [26]:
with open("../data/lensed/all_lensed/galaxies.csv", 'a') as file:
    radec.to_csv(file, index=False, header=False)

# Aggregate outputs

In [1]:
import glob
import pandas as pd
import numpy as np

In [2]:
filenames = glob.glob('../data/testrun/[0-9]*.csv')

In [3]:
data = []
for file in filenames:
    data.append(pd.read_csv(file))

data = pd.concat(data, ignore_index = True)

In [4]:
data

,ls_id,lensed
0,8796115184321665,1
1,8796115089953404,0
2,8796115089888045,1
3,8796115089887865,0
4,8796115278562997,0
...,...,...
968,8796115184323951,0
969,8796115184324356,1
970,8796115278564472,0
971,8796115089888210,0


In [5]:
np.sum(data['lensed'])

568

In [6]:
data.to_csv('../data/testrun/total_lensed.csv', index=False)

In [1]:
from util import collect_lensed, collect_gals

In [2]:
data = collect_lensed('../data/testrun')

Total lensed: 568 / 973


In [4]:
data = collect_lensed('../data/lensed/ML_A')

Total lensed: 78 / 199


In [5]:
data = collect_lensed('../data/lensed/ML_B')

Total lensed: 26 / 56


In [6]:
data = collect_lensed('../data/lensed/ML_C')

Total lensed: 34 / 62


In [7]:
data = collect_lensed('../data/lensed/Huang21_A')

Total lensed: 106 / 206


In [8]:
data = collect_lensed('../data/lensed/Huang21_B')

Total lensed: 117 / 191


In [9]:
data = collect_lensed('../data/lensed/Huang21_C')

Total lensed: 572 / 843


In [2]:
data = collect_gals('../data/testrun')

In [5]:
data

,ls_id,ra,dec,type,dered_mag_g,dered_mag_r,dered_mag_z,dered_mag_w1,dered_mag_w2,unc_g,...,unc_w2,z_phot_median,z_phot_std,z_spec,dered_flux_g,dered_flux_r,dered_flux_z,dered_flux_w1,dered_flux_w2,tag
0,8796115089951731,149.841244,1.018637,DEV,23.201754,22.356958,22.289858,23.750008,NaN,0.065150,...,-0.690722,0.432835,0.178050,-99.0000,0.523961,1.140819,1.213548,0.316226,-2.148673,tst
1,8796115278562038,149.421547,1.406107,DEV,21.702497,21.372538,21.452879,NaN,NaN,0.045546,...,-0.140871,0.231661,0.239397,-99.0000,2.084496,2.824783,2.623304,-1.900767,-15.072433,tst
2,8796115184194653,149.375348,1.374884,DEV,24.719746,23.504185,21.305895,19.629570,20.209246,0.249244,...,0.177981,0.999594,0.104023,-99.0000,0.129450,0.396575,3.003602,14.066046,8.247109,tst
3,8796115278562014,149.376562,1.402851,DEV,25.426490,23.366247,21.207819,19.580494,20.832440,0.670884,...,0.327308,0.956415,0.093704,-99.0000,0.067516,0.450299,3.287548,14.716435,4.645409,tst
4,8796115278498251,149.219260,1.492591,DEV,22.977180,23.026808,22.552841,22.025452,20.894102,0.048105,...,0.325019,1.210540,0.341453,-99.0000,0.644359,0.615569,0.952497,1.548173,4.388934,tst
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1089,8796115184258833,149.580977,1.301445,DEV,24.912098,23.390469,21.305450,19.205685,19.941689,0.237642,...,0.133211,0.972528,0.108875,-99.0000,0.108433,0.440364,3.004832,20.783869,10.551753,tst
1090,8796115184255958,149.635783,1.165474,DEV,24.445368,22.180323,20.618673,19.200708,20.230045,0.245999,...,0.183343,0.708734,0.040529,-99.0000,0.166668,1.342366,5.656280,20.879320,8.090620,tst
1091,8796115278564395,149.492409,1.509756,DEV,23.117222,21.776560,20.934420,19.321192,19.791760,0.104514,...,0.131365,0.594347,0.080693,0.6576,0.566384,1.947045,4.228945,18.686295,12.114234,tst
1092,8796115089952319,149.813087,1.044961,DEV,19.671507,19.266603,19.040766,19.442953,20.152079,0.004291,...,0.175434,0.088458,0.035541,-99.0000,13.533103,19.649826,24.193240,16.703941,8.692971,tst
